<img src="images/dask_horizontal.svg" align="right" width="30%">

# 惰性执行

在这里，我们将讨论dask背后的一些概念，以及代码的惰性执行。 如果您急切希望继续学习本教程，则无需阅读这些材料，但它可能有助于理解dask的基础概念、这些内容如何与您可能已经在使用的技术相适应，以及理解错误会如何发生。

## 序幕

作为 Python 程序员，您可能已经执行了某些*技巧* 来启用大于内存数据集的计算、并行执行或延迟/后台执行。也许用这种说法不够清楚，但一些例子应该会使事情更清楚。 Dask 的重点是让简单的事情变得简单，让复杂的事情成为可能！

除了[详细介绍](http://dask.pydata.org/en/latest/)，我们可以总结一下Dask的基础知识如下：

- 通过将数据分成块并指定任务链来处理不适合内存的数据
- 跨内核甚至集群节点并行执行任务
- 将计算移至数据而不是相反，以最小化通信开销

所有这些都可以让您充分利用计算资源，但仍以一种非常熟悉的方式进行编程：用于构建基本任务的 for 循环、Python 迭代器以及NumPy（Array）多维数组和 Pandas（DataFrame）表格数据。

本笔记本的其余部分将带您了解这些编程范式中的第一个。这可能对于一些用户来说的太过细节，他们可以跳到迭代器、数组和Dataframe部分；但是会有一些数据处理任务不容易适应这些抽象，需要回退到这里的方法。

我们在笔记本的末尾包含了一些示例，表明 Dask 的构建方式背后的想法实际上并不那么新颖，并且有经验的程序员之前会在其他情况下遇到过部分设计。这些例子留给感兴趣的人。

## Dask 是一个图执行引擎

Dask 允许您为要执行的计算构建一个规划方案。 这听起来可能很奇怪，但一个简单的例子将证明您可以在使用非常普通的 Python 函数和 for 循环进行编程时实现这一点。 我们在之前的笔|记本中看到了这一点。

In [ ]:
from dask import delayed

@delayed
def inc(x):
    return x + 1

@delayed
def add(x, y):
    return x + y

在这里，我们使用了delayed修饰来表明我们希望这些函数延迟运行——保存输入集并仅按需执行。 `dask.delayed` 也是一个可以做到这一点的函数，没有修饰，保持原始函数不变，例如，
```python
    delayed_inc = delayed(inc)
```

In [ ]:
# 这看起来像普通代码
x = inc(15)
y = inc(30)
total = add(x, y)
# x、y 和 total 都是delayed对象。
# 它们包含如何进行计算的方案

调用延迟函数会创建一个可以交互检查的delayed对象 (`x, y, total`)。 制作这些对象在某种程度上等同于诸如 `lambda` 或函数包装器之类的构造（见下文）。 每个都有一个描述任务图的简单字典，一个关于如何执行计算的完整规范。

我们可以将对象`total`对应的计算链可视化如下； 圆圈是函数，矩形是数据/结果。

In [ ]:
total.visualize()

但到目前为止，还没有实际执行任何功能。 这展示了 Dask 的图形创建部分（在本例中为`delayed()`）和 Dask 的图形执行部分之间的划分。

要在可视化中运行"图"并实际获得结果，请执行以下操作：

In [ ]:
# 执行所有任务
total.compute()

**你为什么要关心这个？**

通过在执行任何操作之前构建我们想要执行的计算规范，我们可以将规范传递给*执行引擎*进行评估。在 Dask 的情况下，这个执行引擎可以在集群的许多节点上运行，因此您可以访问所有机器上的全部 CPU 内核和内存。 Dask 将智能地执行您的计算，以尽量减少内存中保存的数据量，同时并行化构成图形的任务。请注意，在下面的动画图中，四个worker正在处理（简单）图，执行首先垂直向上进行分支，以便在移动到新分支之前可以删除中间结果。

使用`delayed` 和普通的pythonic 循环代码，可以构建非常复杂的图形并将其传递给Dask 执行。查看[模拟复杂ETL](https://blog.dask.org/2017/01/24/dask-custom) 工作流程的一个很好的例子。

![this](images/grid_search_schedule.gif)

### 练习

我们将`delayed`应用于实际的数据处理任务，尽管是一个简单的任务。

考虑使用 `pd.read_csv` 读取三个 CSV 文件，然后测量它们的总长度。 我们将考虑如何使用普通 Python 代码执行此操作，然后使用延迟为该过程构建一个图，最后使用 Dask 执行该图，以获得超过2倍加速（只有三个输入需要被并行化）。

In [ ]:
%run prep.py -d accounts

In [ ]:
import pandas as pd
import os
filenames = [os.path.join('data', 'accounts.%d.csv' % i) for i in [0, 1, 2]]
filenames

In [ ]:
%%time

# 普通的串行代码
a = pd.read_csv(filenames[0])
b = pd.read_csv(filenames[1])
c = pd.read_csv(filenames[2])

na = len(a)
nb = len(b)
nc = len(c)

total = sum([na, nb, nc])
print(total)

您的任务是使用原始 Python 代码上的delayed函数再次重新创建此图。 你想延迟的三个函数是`pd.read_csv`、`len`和`sum`..

```python
delayed_read_csv = delayed(pd.read_csv)
a = delayed_read_csv(filenames[0])
...

total = ...

# execute
%time total.compute()   
```

In [ ]:
# 你的详细代码写在这里

接下来，使用循环重复此操作，而不是写出所有变量。

In [ ]:
# 你的简洁代码写在这里

In [ ]:
## 详细版本
delayed_read_csv = delayed(pd.read_csv)
a = delayed_read_csv(filenames[0])
b = delayed_read_csv(filenames[1])
c = delayed_read_csv(filenames[2])

delayed_len = delayed(len)
na = delayed_len(a)
nb = delayed_len(b)
nc = delayed_len(c)

delayed_sum = delayed(sum)

total = delayed_sum([na, nb, nc])
%time print(total.compute())


## 简介版本
csvs = [delayed(pd.read_csv)(fn) for fn in filenames]
lens = [delayed(len)(csv) for csv in csvs]
total = delayed(sum)(lens)
%time print(total.compute())


**笔记**

Delayed对象支持各种操作：
```python
    x2 = x + 1
```

如果 `x` 是delayed结果（如上面的 `total`），那么 `x2` 也是。 支持的操作包括算术运算符、项目或切片选择、属性访问和方法调用——基本上任何可以被表述为"lambda"表达式的操作。

*不*支持的操作包括变异、setter方法、迭代（for）和布尔值（判断式）。

## 附录：更多细节和示例

以下示例表明，在处理大数据时，Dask 所做的事情与正常的 Python 编程并没有太大区别。 这些示例**仅供专家使用**，普通用户可以继续使用教程中的下一个笔记本。

### 示例 1：简单的字数统计

该目录包含一个名为`README.md`的文件。 您将如何计算该文件中的单词数？

最简单的方法是将所有数据加载到内存中，在空白处拆分并计算结果数量。 这里我们使用正则表达式来拆分单词。

In [ ]:
import re
splitter = re.compile('\w+')
with open('README.md', 'r') as f:
    data = f.read()
result = len(splitter.findall(data))
result

这种方法的问题在于它不能扩展——如果文件非常大，它和生成的单词列表可能会填满内存。 我们可以很容易地避免这种情况，因为我们只需要一个简单的总和，并且每一行都完全独立于其他行。 现在我们评估每条数据并立即再次释放空间，以便我们可以对任意大的文件执行此操作。 请注意，时间效率和内存占用之间通常存在权衡：以下使用很少的内存，但对于未填充大量内存的文件可能会更慢。 一般来说，人们希望块足够小，不会给内存带来压力，但足够大以有效使用 CPU。

In [ ]:
result = 0
with open('README.md', 'r') as f:
    for line in f:
        result += len(splitter.findall(line))
result

### 示例2：后台执行

有许多任务需要一段时间才能完成，但实际上并不需要太多的 CPU，例如任何需要通过网络进行通信或来自用户输入的任务。 在典型的串行编程中，需要在进程完成时停止执行，然后继续执行。 这对用户体验来说是可怕的（想象一下缓慢的进度条会锁定应用程序并且无法取消），并且浪费时间（同时 CPU 可能一直在做有用的工作）。

例如，我们可以按如下方式启动进程并获取它们的输出：
```python
    import subprocess
    p = subprocess.Popen(command, stdout=subprocess.PIPE)
    p.returncode
```

任务在单独的进程中运行，返回码将保持为`None`，直到完成，那时它将被改为`0`时。 要返回结果，我们需要 `out = p.communicate()[0]`（如果进程未完成，它将阻塞）。

同样，我们可以在后台启动 Python 进程和线程。 有些方法允许映射多个输入并收集结果，稍后会详细介绍。 线程启动并且单元格立即完成，但与下载相关的数据仅在一段时间后出现在队列对象中。

In [ ]:
# 编辑 sources.py 以配置源位置
import sources
sources.lazy_url

In [ ]:
import threading
import queue
import urllib

def get_webdata(url, q):
    u = urllib.request.urlopen(url)
    # raise ValueError
    q.put(u.read())

q = queue.Queue()
t = threading.Thread(target=get_webdata, args=(sources.lazy_url, q))
t.start()

In [ ]:
# 将结果取回此线程。 如果工作线程没有完成，这将等待。
q.get()

考虑：如果`get_webdata` 函数中出现异常，你会看到什么？ 您可以取消注释上面的 `raise` 行，然后重新执行这两个单元格。 发生什么了？ 有什么方法可以调试执行以找到错误的根本原因吗？

### 示例 3：延迟执行

Python 中有很多方法可以指定要执行的计算，但只能*稍后*运行。

In [ ]:
def add(x, y):
    return x + y

# 有时我们使用字符串推迟计算
x = 15
y = 30
z = "add(x, y)"
eval(z)

In [ ]:
# 我们可以使用 lambda 或其他“闭包”
x = 15
y = 30
z = lambda: add(x, y)
z()

In [ ]:
# functools.partial 中发生了非常相似的事情

import functools
z = functools.partial(add, x, y)
z()

In [ ]:
# Python 生成器默认延迟执行
# 许多 Python 函数都期望这样的可迭代对象

def gen():
    res = x
    yield res
    res += y
    yield res

g = gen()

In [ ]:
# 运行一次：我们得到一个值并在生成器中停止执行
# 再次运行，执行完成
next(g)

### Dask 任务图

任何 Dask 对象，例如上面的`total`，都有一个属性来描述产生该结果所需的计算。 确实，这正是我们一直在谈论的图，它可以被可视化。 我们看到它是一个简单的字典，其中键是唯一的任务标识符，值是计算的函数和输入。

`delayed` 是创建 Dask 任务图的一种方便的机制，但喜欢冒险的人可能希望通过直接构建图词典来充分发挥灵活性。 详细信息可以在[这里](http://dask.pydata.org/en/latest/graphs.html)找到。

In [ ]:
total.dask

In [ ]:
dict(total.dask)